In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from datetime import datetime, timedelta
from pytz import timezone
from safe import sql_password, weather_api_key, rapidapi_key #.py file with keys etc.
import sqlalchemy
import pymysql

# Get City Data Via Webscraping

## Get City Data (incl. Latitude & Longitude)

In [2]:
# City DataFrame
cities = ['Berlin','Hamburg','Munich']

city_data =[]

for city in cities:
    url = str(f'https://en.wikipedia.org/wiki/{city}')
    response = requests.get(url)
    soup_c = BeautifulSoup(response.content, 'html.parser')

    country = soup_c.find(class_ = 'infobox-data').get_text()
    latitude = soup_c.find(class_ = 'latitude').get_text()
    longitude = soup_c.find(class_ = 'longitude').get_text()

    city_data.append({'city':city,
                      'country':country,
                      'latitude':parse(latitude),       # convert to decimal ( with parse)
                      'longitude':parse(longitude)})    # convert to decimal ( with parse)
    

cities_df = pd.DataFrame(city_data)
cities_df

,city,country,latitude,longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


## Get Population Data

In [3]:
# Population DataFrame

population_data = []

for city in cities:
    url = str(f'https://en.wikipedia.org/wiki/{city}')
    response = requests.get(url)
    soup_c = BeautifulSoup(response.content, 'html.parser')

    population = (soup_c.find('table', class_ = 'vcard')
                  .find(string = 'Population')
                  .find_next('td')
                  .get_text())
    population_clean = population.replace(',','')                   # clean data to make it readable as an integer
    date = datetime.today().strftime('%Y')                          # convert date to datetime datatype

    population_data.append({'city':city,
                            'population':int(population_clean),     # convert population to integer
                            'year_data':date})
    
population_df = pd.DataFrame(population_data)
population_df

,city,population,year_data
0,Berlin,3878100,2024
1,Hamburg,1964021,2024
2,Munich,1510378,2024


## Send Data to SQL

In [4]:
# Connect to SQL

schema = 'cities'
host = '127.0.0.1'
user = 'root'
password = sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [5]:
# Send Cities DataFrame to SQL

cities_df.to_sql('city',
                 if_exists='append',
                 con=connection_string,
                 index=False)

3

In [6]:
# Get city from SQL for foreign key in population

city_from_sql = pd.read_sql('city', con=connection_string)
city_from_sql

,city_id,city,country,latitude,longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


In [7]:
# Prepare Population DataFrame to be send to SQL

# Add city ID (for foreign key in SQL)
population_df = population_df.merge(city_from_sql[['city_id','city']],
                          on ='city',
                          how='left')

# Drop unnecessary columns
population_df = population_df.drop(columns=['city'])

population_df

,population,year_data,city_id
0,3878100,2024,1
1,1964021,2024,2
2,1510378,2024,3


In [8]:
# Send Population DataFrame to SQL
population_df.to_sql('population',
                 if_exists='append',
                 con=connection_string,
                 index=False)

3

# Get Weather Data via API (openWeather)

In [27]:
# Function to retrieve weather data 

def retrieval_and_send_data():
    connection_string = create_connection_string()
    cities_df = fetch_cities_data(connection_string)
    weather_df = retrieve_weather_data(cities_df)
    store_weather_data(weather_df,connection_string)
    return 'Data updated'

def create_connection_string():
    schema = 'cities'
    host = '127.0.0.1'
    user = 'root'
    password = sql_password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
    return pd.read_sql('city', con=connection_string)


def retrieve_weather_data(cities_df):
    weather_data = {'forecast_time':[],
                           'descr':[],
                           'temperature':[],
                           'feels_like':[],
                           'humidity':[],
                           'wind':[],
                           'weatherstation':[],
                           'time_retrieved':[],
                           'city_id':[]}
   

    # get api connection
    for _,c in cities_df.iterrows():

        #get JSON 
        url = 'https://api.openweathermap.org/data/2.5/forecast'
        latitude = c['latitude']   # (cities_df.loc[cities_df['city'] == c,'latitude'])
        longitude = c['longitude'] # (cities_df.loc[cities_df['city'] == c,'longitude']).astype('float')
        city_id = c['city_id'] # (cities_df.loc[cities_df['city'] == c,'city_id'])
        appid = weather_api_key
        units = 'metric'
        querystring = {'lat':latitude,'lon':longitude,'appid':appid, 'units':units}

        weather = requests.request('GET', url , params=querystring)
        weather_json = weather.json()

        # get DataFrame
        forecast = weather_json['cnt']
        
        for fc in range(forecast):
            weather_data['forecast_time'].append(weather_json['list'][fc]['dt_txt'])
            weather_data['descr'].append(weather_json['list'][fc]['weather'][0]['description'])
            weather_data['temperature'].append(weather_json['list'][0]['main']['temp'])
            weather_data['feels_like'].append(weather_json['list'][0]['main']['feels_like'])
            weather_data['humidity'].append(weather_json['list'][0]['main']['humidity'])
            weather_data['wind'].append(weather_json['list'][0]['wind']['speed'])
            weather_data['weatherstation'].append(weather_json['city']['name'])
            weather_data['time_retrieved'].append(datetime.today().strftime('%d-%m-%Y %H:%M:%S'))
            weather_data['city_id'].append(city_id)

    weather_df = pd.DataFrame(weather_data)
    weather_df['forecast_time'] = pd.to_datetime(weather_df['forecast_time'])
    weather_df['time_retrieved'] = pd.to_datetime(weather_df['time_retrieved'])

    
    return weather_df 

def store_weather_data(weather_df,connection_string):
    weather_df.to_sql('weather', if_exists='append', con= connection_string, index=False)

In [28]:
# Run function
retrieval_and_send_data()

'Data updated'

# Get Flight Info via API

## Retrieve Airport Info

In [29]:
# Cet longitudes & latitudes from SQL
# connect to SQL
schema = 'cities'
host = '127.0.0.1'
user = 'root'
password = sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# get cities table
cities_sql = pd.read_sql('city', con=connection_string)

#latitudes & longitudes
latitude = list(cities_sql['latitude'])
longitude = list(cities_sql['longitude'])

In [30]:
# Retrieval function
def icao_airport_codes(latitudes, longitudes):
  
  list_for_df = []

  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/10"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": rapidapi_key
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

In [31]:
# Run function for cities from SQL
latitudes = latitude
longitudes = longitude

city_airports = icao_airport_codes(latitudes, longitudes)

In [32]:
# Prep table to push to SQL
city_airports = city_airports.merge(cities_sql[['city_id','city']],
                                    how='left',
                                    left_on='municipalityName',
                                    right_on = 'city')
# Condense to important info
city_airport = city_airports[['icao','iata','name','city_id']]
city_airport = city_airport.rename(columns={'name':'airport_name'},errors='raise')

In [33]:
# Send to sql
city_airport.to_sql('airport',
                    if_exists='append',
                    con=connection_string,
                    index=False)

3

## Retrieve Flight Data

In [36]:
def retrieval_and_send_data_flights():
    connection_string = connection()
    airport_df = fetch_airport_data(connection_string)
    flight_df = retrieve_flight_data(airport_df)
    store_flight_data(flight_df,connection_string)
    return 'Data updated'

def connection():
    schema = 'cities'
    host = '127.0.0.1'
    user = 'root'
    password = sql_password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_airport_data(connection_string):
    return pd.read_sql('airport', con=connection_string)

def retrieve_flight_data(airport_df):
    berlin_timezone = timezone('Europe/Berlin')
    #today = datetime.today()
    tomorrow = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
    T1 =['00:00','12:00']
    T2 =['11:59','23:59']
    
    flights =[]
    for index,value in enumerate(T1):

        for icao in airport_df['icao']:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{value}/{tomorrow}T{T2[index]}"

            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

            headers = {
                "x-rapidapi-key": rapidapi_key,
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)
            flight_json = response.json()

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flight_json['arrivals']:
                flight ={
                    'icao': icao,
                    'scheduled_arrival':item['arrival']['scheduledTime'].get('local',None),
                    'terminal':item['arrival'].get('terminal',None),
                    'baggage_belt':item['arrival'].get('baggageBelt', 'not assigned'),
                    'departure_airport':item['departure']['airport'].get('name',None),
                    'flight_number':item.get('number',None),
                    'airline':item['airline'].get('name',None),
                    'data_retrieval_time': retrieval_time
                }

                flights.append(flight)

    flight_df = pd.DataFrame(flights)
    flight_df["scheduled_arrival"] = flight_df["scheduled_arrival"].str[:-6]
    flight_df["scheduled_arrival"] = pd.to_datetime(flight_df["scheduled_arrival"])
    flight_df['data_retrieval_time'] = pd.to_datetime(flight_df['data_retrieval_time'])

    return flight_df

def store_flight_data(flight_df,connection_string):
    flight_df.to_sql('flight', if_exists='append', con=connection_string, index=False)

In [37]:
# Run function
retrieval_and_send_data_flights()

'Data updated'